## Convert TCDF results to bayesian network

In [ ]:
import bnlearn as bn
import numpy as np
import pickle as pkl
import pandas as pd
import mlflow
import mlflow.sklearn

import sys
import numpy

import pyAgrum as gum
import pyAgrum.skbn as skbn
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.image as gph 
import pyAgrum.lib.bn2graph as bng
import pyAgrum.lib.bn_vs_bn as compare
import pyAgrum.lib.bn_vs_bn as compare
import pydotplus
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
numpy.set_printoptions(threshold=sys.maxsize)

%matplotlib inline

### Result 1: selected features, all parts

In [ ]:
tcdf_edges = [('Taktzeit gesamt', 'Durchmesser Min. Fübo'), ('Taktzeit Abrichten OP1 Sp. 51', 'Durchmesser Min. Fübo'),
              ('Be- / Entladezeit Seite A', 'Taktzeit gesamt'), ('Anz. Abrichthübe Sp. 51', 'Be- / Entladezeit Seite A'),
              ('Anzahl Teile seit letztem Scheibenwechsel Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'), 
              ('Anzahl Teile seit letztem Scheibenwechsel Sp. 51', 'Unwucht Sp. 51 Leerlauf'),
              ('Leistung Sp. 51', 'Aktueller Durchmesser Sp. 51'), ('Korr. Fübo 1 Sp. 51', 'Aktueller Durchmesser Sp. 51'),
              ('Aktueller Durchmesser Sp. 51', 'Anzahl Teile seit letztem Scheibenwechsel Sp. 51'),
              ('Anzahl Teile seit letztem Scheibenwechsel Sp. 51','Anz. Abrichthübe Sp. 51'),
              ('Unwucht Sp. 51 Leerlauf', 'Anz. Abrichthübe Sp. 51'), 
              ('Anz. Abrichthübe Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'),
              ('Korr. Fübo 1 Sp. 51', 'Anzahl Teile seit letztem Scheibenwechsel Sp. 51'),
              ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Unwucht Sp. 51 Leerlauf'),
              ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo'),
              ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Ext. Korr. Fübo 1 Sp. 51'),
              ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Ext. Durchmesser Min. Fübo 51'),
              ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Taktzeit Schleifen OP1 Sp. 51'),
              ('Anzahl Teile seit letztem Abrichten Sp. 51','Taktzeit gesamt'),
              ('Korr. Fübo 1 Sp. 51', 'Anzahl Teile seit letztem Abrichten Sp. 51'),
              ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'),
              ('Taktzeit Abrichten OP1 Sp. 51', 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51'),
              ('Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo', 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51'),
              ('Aktueller Durchmesser Sp. 51', 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo'),
              ('Korr. Fübo 1 Sp. 51', 'Unwucht Sp. 51 Leerlauf'), 
              ('Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo', 'Unwucht Sp. 51 Leerlauf'),
              ('Korr. Fübo 1 Sp. 51', 'Taktzeit gesamt'), ('Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo', 'Taktzeit gesamt'),
              ('Taktzeit Schleifen OP1 Sp. 51', 'Taktzeit gesamt'), 
              ('Taktzeit Abrichten OP1 Sp. 51', 'Taktzeit gesamt'), 
              ('Korr. Fübo 1 Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'), ('Korr. PSH Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'),
              ('Taktzeit Abrichten OP1 Sp. 51', 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf'),
              #('Taktzeit Abrichten OP1 Sp. 51', 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo'), 
              ('Stückzahl bis Scheibenwechsel Sp. 51', 'Leistung Sp. 51'), ('Stückzahl bis Scheibenwechsel Sp. 51', 'Korr. PSH Sp. 51'),
              ('Korr. Fübo 1 Sp. 51', 'Stückzahl bis Scheibenwechsel Sp. 51'),
              ('Leistung Sp. 51', 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf'), 
              ('Korr. Fübo 1 Sp. 51', 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf'),
              ('PSH Pos. B5', 'Korr. PSH Sp. 51'), ('Leistung Sp. 51', 'PSH Pos. B5'),
              ('Korr. Fübo 1 Sp. 51', 'PSH Pos. B5'), ('Leistung Sp. 51', 'Korr. PSH Sp. 51'),
              ('Korr. Fübo 1 Sp. 51', 'Leistung Sp. 51'), ('Korr. PSH Sp. 51', 'Ext. Durchmesser Min. Fübo 51'),
              ('Korr. Fübo 1 Sp. 51', 'Korr. PSH Sp. 51'), ('Ext. Durchmesser Min. Fübo 51', 'Ext. Korr. Fübo 1 Sp. 51')]

### Results 2: selected features, equal number of scrap and non-scrap parts

In [ ]:
tcdf_edges = [
('Korr. Fübo 1 Sp. 51', 'Aktueller Durchmesser Sp. 51' ),
('Anz. Abrichthübe Sp. 51' , 'Aktueller Durchmesser Sp. 51' ),
('Anzahl Teile seit letztem Abrichten Sp. 51' , 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51' ),
('Taktzeit gesamt' , 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51' ),
('Anz. Abrichthübe Sp. 51' , 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51' ),
('Taktzeit Abrichten OP1 Sp. 51' , 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51' ),
('Stückzahl bis Scheibenwechsel Sp. 51' , 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51' ),
('PSH Pos. B5' , 'Anz. ext. Korrektur PSH Fübo 1 Sp. 51' ),
('Taktzeit gesamt' , 'Anzahl Teile seit letztem Abrichten Sp. 51' ),
('Taktzeit Abrichten OP1 Sp. 51' , 'Anzahl Teile seit letztem Abrichten Sp. 51' ),
('Leistung Sp. 51' , 'Anzahl Teile seit letztem Abrichten Sp. 51' ),
('Korr. PSH Sp. 51' , 'Anzahl Teile seit letztem Abrichten Sp. 51' ),
('Stückzahl bis Scheibenwechsel Sp. 51' , 'Anzahl Teile seit letztem Abrichten Sp. 51' ),
('Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo' , 'Anzahl Teile seit letztem Abrichten Sp. 51' ),
('Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' , 'Anzahl Teile seit letztem Abrichten Sp. 51' ),
('Korr. Fübo 1 Sp. 51' , 'Anzahl Teile seit letztem Scheibenwechsel Sp. 51' ),
('Taktzeit gesamt' , 'Be- / Entladezeit Seite A' ),
('Korr. Fübo 1 Sp. 51' , 'Ext. Durchmesser Min. Fübo 51' ),
('Anz. Abrichthübe Sp. 51' ,'Ext. Durchmesser Min. Fübo 51' ),
('Anz. ext. Korrektur PSH Fübo 1 Sp. 51' , 'Ext. Durchmesser Min. Fübo 51' ),
('Ext. Durchmesser Min. Fübo 51' , 'Ext. Korr. Fübo 1 Sp. 51' ),
('Stückzahl bis Scheibenwechsel Sp. 51' , 'Ext. Korr. Fübo 1 Sp. 51' ),
('Korr. PSH Sp. 51' , 'Korr. Fübo 1 Sp. 51' ),
('PSH Pos. B5' , 'Korr. Fübo 1 Sp. 51' ),
('PSH Pos. B5' , 'Korr. PSH Sp. 51' ),
('Anz. Abrichthübe Sp. 51' , 'Korr. PSH Sp. 51' ),
('Stückzahl bis Scheibenwechsel Sp. 51' , 'Leistung Sp. 51' ),
('Anzahl Teile seit letztem Scheibenwechsel Sp. 51' , 'Leistung Sp. 51' ),
('Taktzeit gesamt' , 'Leistung Sp. 51' ),
('Korr. Fübo 1 Sp. 51' , 'Leistung Sp. 51' ),
('Anz. Abrichthübe Sp. 51' , 'PSH Pos. B5' ),
('Unwucht Sp. 51 Leerlauf' , 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' ),
('Taktzeit gesamt' , 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' ),
('PSH Pos. B5' , 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' ),
('Anz. Abrichthübe Sp. 51' , 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' ),
('Stückzahl bis Scheibenwechsel Sp. 51' , 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' ),
('Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' , 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo' ),
('Taktzeit Abrichten OP1 Sp. 51' , 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo' ),
('Korr. Fübo 1 Sp. 51' , 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo' ),
('Be- / Entladezeit Seite A' , 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo' ),
('Aktueller Durchmesser Sp. 51' , 'Stückzahl bis Scheibenwechsel Sp. 51' ),
('Anz. Abrichthübe Sp. 51' , 'Stückzahl bis Scheibenwechsel Sp. 51' ),
('Korr. PSH Sp. 51' , 'Taktzeit Abrichten OP1 Sp. 51' ),
('Anz. Abrichthübe Sp. 51' , 'Taktzeit Abrichten OP1 Sp. 51'),
('PSH Pos. B5' , 'Taktzeit Abrichten OP1 Sp. 51' ),
('Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf' , 'Taktzeit Abrichten OP1 Sp. 51' ),
('PSH Pos. B5' , 'Taktzeit Schleifen OP1 Sp. 51' ),
('Taktzeit gesamt' , 'Taktzeit Schleifen OP1 Sp. 51' ),
('Durchmesser Min. Fübo' , 'Taktzeit Schleifen OP1 Sp. 51' ),
('Anzahl Teile seit letztem Abrichten Sp. 51' , 'Taktzeit Schleifen OP1 Sp. 51' ),
('Taktzeit Abrichten OP1 Sp. 51' , 'Taktzeit gesamt' ),
('Be- / Entladezeit Seite A' , 'Taktzeit gesamt' ),
('Anz. Abrichthübe Sp. 51' , 'Taktzeit gesamt' ),
('Aktueller Durchmesser Sp. 51' , 'Taktzeit gesamt' ),
('Durchmesser Min. Fübo' , 'Taktzeit gesamt' ),
('PSH Pos. B5' , 'Unwucht Sp. 51 Leerlauf '),
('Anzahl Teile seit letztem Abrichten Sp. 51' , 'Unwucht Sp. 51 Leerlauf' ),
('Aktueller Durchmesser Sp. 51' , 'Unwucht Sp. 51 Leerlauf' ),
('Taktzeit gesamt' ,' Unwucht Sp. 51 Leerlauf' ),
('Taktzeit Abrichten OP1 Sp. 51' , 'Durchmesser Min. Fübo' ),
]

In [ ]:
import pyAgrum.lib.notebook as gnb

bn8=gum.BayesNet('TCDF-ScrapAnalysis')
#counter = 1
nodes = []
for cols in normalized_df.columns:
    node = bn8.add(gum.LabelizedVariable(str(cols), str(cols)))
    nodes.append(node)

for link in tcdf_edges:
    print(link)
    bn8.addArc(*link)

gnb.show(bn8,size="15!")

In [ ]:
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.image as gph 
import pyAgrum.lib.bn2graph as bng

dot_data = bng.BN2dot(bn8,size="10")
dot_data.set_size('"20,20!"')
dot_data.write_png('TCDF-BN-all-parts.png')

In [ ]:
##compare tcdf with ground truth data
diff = compare.GraphicalBNComparator(bn,bn8).scores()
# get tp, tn, fp, fn to draw the confusion matrix 
json_str = json.dumps(diff)

#load the json to a string
resp = json.loads(json_str)

tp1 = resp['count']['tp']
tn1 = resp['count']['tn']
fp1 = resp['count']['fp']
fn1 = resp['count']['fn']

In [ ]:
#result of all data 
#{'count': {'tp': 7, 'tn': 321, 'fp': 27, 'fn': 25}, 'recall': 0.21875, 'precision': 0.20588235294117646, 'fscore': 0.21212121212121213, 'dist2opt': 1.1139903051060374} {'count': {'tp': 9, 'tn': 321, 'fp': 27, 'fn': 23}, 'recall': 0.28125, 'precision': 0.25, 'fscore': 0.2647058823529412, 'dist2opt': 1.0387981336621663} {'count': {'tp': 11, 'tn': 324, 'fp': 24, 'fn': 21}, 'recall': 0.34375, 'precision': 0.3142857142857143, 'fscore': 0.3283582089552239, 'dist2opt': 0.9491407398972258}

#result of first parts
#{'count': {'tp': 3, 'tn': 298, 'fp': 16, 'fn': 25}, 'recall': 0.10714285714285714, 'precision': 0.15789473684210525, 'fscore': 0.1276595744680851, 'dist2opt': 1.2273284612479445} {'count': {'tp': 7, 'tn': 302, 'fp': 12, 'fn': 21}, 'recall': 0.25, 'precision': 0.3684210526315789, 'fscore': 0.2978723404255319, 'dist2opt': 0.9805059748716489} {'count': {'tp': 3, 'tn': 305, 'fp': 9, 'fn': 25}, 'recall': 0.10714285714285714, 'precision': 0.25, 'fscore': 0.15, 'dist2opt': 1.1660591226653219}

#result of last parts
#{'count': {'tp': 2, 'tn': 270, 'fp': 12, 'fn': 22}, 'recall': 0.08333333333333333, 'precision': 0.14285714285714285, 'fscore': 0.10526315789473685, 'dist2opt': 1.2549787469629907} {'count': {'tp': 3, 'tn': 271, 'fp': 11, 'fn': 21}, 'recall': 0.125, 'precision': 0.21428571428571427, 'fscore': 0.15789473684210525, 'dist2opt': 1.1759982732876397} {'count': {'tp': 2, 'tn': 270, 'fp': 12, 'fn': 22}, 'recall': 0.08333333333333333, 'precision': 0.14285714285714285, 'fscore': 0.10526315789473685, 'dist2opt': 1.2549787469629907}

#result of scrap parts
#{'count': {'tp': 6, 'tn': 334, 'fp': 14, 'fn': 26}, 'recall': 0.1875, 'precision': 0.3, 'fscore': 0.23076923076923075, 'dist2opt': 1.0724533789400823} {'count': {'tp': 4, 'tn': 330, 'fp': 18, 'fn': 28}, 'recall': 0.125, 'precision': 0.18181818181818182, 'fscore': 0.14814814814814814, 'dist2opt': 1.1979342584646728} {'count': {'tp': 4, 'tn': 334, 'fp': 14, 'fn': 28}, 'recall': 0.125, 'precision': 0.2222222222222222, 'fscore': 0.16, 'dist2opt': 1.1707105840492509}

#result of equal number of parts
#{'count': {'tp': 3, 'tn': 327, 'fp': 21, 'fn': 29}, 'recall': 0.09375, 'precision': 0.125, 'fscore': 0.10714285714285714, 'dist2opt': 1.2597277731716483} {'count': {'tp': 3, 'tn': 327, 'fp': 21, 'fn': 29}, 'recall': 0.09375, 'precision': 0.125, 'fscore': 0.10714285714285714, 'dist2opt': 1.2597277731716483} {'count': {'tp': 4, 'tn': 333, 'fp': 15, 'fn': 28}, 'recall': 0.125, 'precision': 0.21052631578947367, 'fscore': 0.1568627450980392, 'dist2opt': 1.1785133423347154}

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

y_true = [tp1, fn1]
y_pred = [fp1, tn1]
labels = ["True Pos","False Neg","False Pos","True Neg"]
categories = ["True", "False"]
cf_matrix = np.column_stack((y_true, y_pred))

make_confusion_matrix(cf_matrix, 
                      group_names=labels,
                      categories=categories)

In [ ]:
#save results
tcdf_diff = compare.GraphicalBNComparator(bn,bn8).dotDiff()
tcdf_diff.write_png("gt-vs-tcdf-all-parts.png")